In [819]:
# IMPORT ------
    #* Dataframe 
import pandas as pd 
    #* Matrices 
import numpy as np
    #* Preprocessing 
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
    #* Splitter 
from sklearn.model_selection import train_test_split

    # Misc
import os 

In [820]:
# PATH ------
# base_path = "/home/jovyan/work"  #"/Maturite_dentaire"
data_path = "data/Teeth"
data_name= "dataset.csv"

# CWD ------
# os.chdir(base_path)

# IMPORT TYPES ------
pandas_types= {'ID':'int','VAL_I1':'category','VAL_I2':'category',
            'VAL_C1':'category','VAL_PM1':'category',
            'VAL_PM2':'category','VAL_M1':'category',
            'VAL_M2':'category','VAL_M3':'category'}

# ORDINAL ENCODING ------
ord_encoding= {'A':2,'B':3,'C':4,'D':5,'E':6,'F':7,'G':8,'H':9}

# RANDOM STATE ------
rs= 12345

# PREPROCESSING 

## IMPORT DATA 

In [821]:
# IMPORT DATA ------
df = pd.read_csv(os.path.join(data_path,data_name),
                 sep=';',
                 dtype=pandas_types)

# Remove useless columns ---
X = df.drop(['ID', 'PAT_AGE'], axis=1)
Y = df['PAT_AGE']

# Train test split 

In [822]:
X_train,X_test, y_train, y_test = train_test_split(X,Y, random_state= rs)  

In [823]:
print(f"Shape Training Data : {X_train.shape}")
print(f"Shape Testing Data : {X_test.shape}")

print(f"Shape Training Target : {y_train.shape}")
print(f"Shape Testing Target : {y_test.shape}")

Shape Training Data : (2135, 9)
Shape Testing Data : (712, 9)
Shape Training Target : (2135,)
Shape Testing Target : (712,)


In [824]:
# Save to csv ---
X_train.to_csv(os.path.join(data_path,"X_train.csv"), index=False)
y_train.to_csv(os.path.join(data_path,"y_train.csv"), index=False)

X_test.to_csv(os.path.join(data_path,"X_test.csv"), index=False)
y_test.to_csv(os.path.join(data_path,"y_test.csv"), index=False)

## Ordinal encoding 

In [825]:
# Simple Ordinal Encoding ---
X_train.replace(ord_encoding, inplace=True)
X_test.replace(ord_encoding, inplace=True)

# Saving Ordinal encoded data --- 
X_train.to_csv(os.path.join(data_path,"1_X_train_encoded.csv"),index=False)
X_test.to_csv(os.path.join(data_path,"1_X_test_encoded.csv"),index=False)

## Removing outliers 

In [826]:
# Remove all samples for which at least 7 out of 8 teeth info is missing --- 
to_rm = np.where(np.sum(pd.isna(X_train),axis=1)>=7)[0] # rowsums : looking for >= 7
X_train = X_train.reset_index(drop=True).drop(to_rm,axis=0) # remove rows from training 
y_train = y_train.reset_index(drop=True).drop(to_rm,axis=0) # remove rows from labels 

# reset indexes --- 
X_train.reset_index(drop= True, inplace=True)
y_train.reset_index(drop= True, inplace=True)

# Printing n° of rows removed : 
print(f"Number of samples removed from trainin data : {to_rm.shape}")

Number of samples removed from trainin data : (118,)


# TRAIN VAL TEST SPLIT 

- Training 
- Validation 
- Testing

In [827]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, random_state= rs,test_size=0.1)

In [828]:
print(f"Shape Training Data : {X_train.shape}")
print(f"Shape Training Target : {y_train.shape}")
print("\n")
print(f"Shape Validation Target : {X_val.shape}")
print(f"Shape Validation Data : {y_val.shape}")
print("\n")
print(f"Shape Testing Data : {X_test.shape}")
print(f"Shape Testing Target : {y_test.shape}")

Shape Training Data : (1815, 9)
Shape Training Target : (1815,)


Shape Validation Target : (202, 9)
Shape Validation Data : (202,)


Shape Testing Data : (712, 9)
Shape Testing Target : (712,)


## Imputing 

In [829]:
# IMPUTING STRATEGY ------
knni= KNNImputer(n_neighbors= 40,weights= "distance",add_indicator= True)
X_train= knni.fit_transform(X_train.values) 
X_val= knni.transform(X_val.values) 
X_test= knni.transform(X_test.values) 

# New columns --- 
indicator_features= []
for i in X.columns.values[1:]:
    indicator_features.append(i+ "_missing_indicator")
new_cols= [*list(X.columns), *indicator_features]

# Saving Ordinal encoded data --- 
pd.DataFrame(X_train,columns= new_cols).to_csv(os.path.join(data_path,"2_X_train_encoded.csv"),index=False)
pd.DataFrame(X_val,columns= new_cols).to_csv(os.path.join(data_path,"2_X_val_encoded.csv"),index=False)
pd.DataFrame(X_test, columns= new_cols).to_csv(os.path.join(data_path,"2_X_test_encoded.csv"),index=False)

## Outlier removal 

In [830]:
# isolation forest 
#train ---
Y_pred_train = IsolationForest(random_state=rs,bootstrap=True,contamination=0.05).fit_predict(X_train) # fit outlier detection # Returns -1 for outliers and 1 for inliers.
inliers = np.where(Y_pred_train == 1)[0] # which samples to keep 
    #* Keep non outliers
X_train = X_train[inliers,:]
y_train = y_train.reset_index(drop=True)[inliers]

In [831]:
print(f"Shape Training Data : {X_train.shape}")
print(f"Shape Training Labels Data : {y_train.shape}")

Shape Training Data : (1726, 17)
Shape Training Labels Data : (1726,)


In [832]:
# Saving preprocesssed data --- 
    #* Train 
pd.DataFrame(X_train,columns= new_cols).to_csv(os.path.join(data_path,"3_X_train_outlier.csv"),index=False)
pd.Series(y_train,name="Age").to_csv(os.path.join(data_path,"3_y_train_outlier.csv"),index=False)
    #* val 
pd.DataFrame(X_val,columns= new_cols).to_csv(os.path.join(data_path,"3_X_val_outlier.csv"),index=False)
pd.Series(y_val,name="Age").to_csv(os.path.join(data_path,"3_y_val_outlier.csv"),index=False)
    #* Train 
pd.DataFrame(X_test,columns= new_cols).to_csv(os.path.join(data_path,"3_X_test_outlier.csv"),index=False)
pd.Series(y_test,name="Age").to_csv(os.path.join(data_path,"3_y_test_outlier.csv"),index=False)

In [833]:
import xgboost as xgb


In [834]:
objective = "reg:squaredlogerror"  # metric RMSLE
objective = "reg:squarederror" # RMSLE STUCKED ...
objective = "reg:gamma"
metric = "mae"
reg = xgb.XGBRegressor(colsample_bytree=0.8, gamma=0.1, max_depth=4,reg_lambda=0.01,n_estimators=100,
              min_child_weight=5, metric=metric,
              objective=objective,subsample=0.8, random_state=rs)
reg.fit(X=X_train,
            y=y_train,
            eval_set=((X_train,y_train),(X_val,y_val)),  # Validation set for early stopping (validates on test -- last data tuple)
            early_stopping_rounds=100,
            verbose=0,
            eval_metric=["mae"]
        )

[20:55:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0.1, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=4, metric='mae', min_child_weight=5,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, objective='reg:gamma', predictor='auto',
             random_state=12345, reg_alpha=0, reg_lambda=0.01,
             scale_pos_weight=None, subsample=0.8, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [835]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(reg.predict(X_test), y_test)

1.177415287521448

In [836]:
import numpy as np
np.min(reg.evals_result_['validation_1']['mae'])

1.045026

In [837]:
import plotly.express as px
px.line(y=[reg.evals_result_['validation_0']['mae'], reg.evals_result_['validation_1']['mae']])

In [844]:
import tensorflow as tf
keras = tf.keras
reg = keras.regularizers.l2(0.01)

model3 = keras.models.Sequential([
    keras.layers.Input(shape=X_train.shape[1:]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer =reg),
    keras.layers.Dropout(0.1),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer =reg),
    keras.layers.Dropout(0.1),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer =reg),
    keras.layers.Dropout(0.1),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])
optimiser = keras.optimizers.Adam(lr=0.001)
model3.compile(loss="mae", optimizer=optimiser, metrics=["mae"])
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 17)                68        
_________________________________________________________________
dense_4 (Dense)              (None, 100)               1800      
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 100)               400       
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 100)              

C:\Users\alexi\AppData\Roaming\Python\Python39\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [845]:
checkpoint = keras.callbacks.ModelCheckpoint("save/model.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)

model3_history = model3.fit(X_train, y_train, epochs=500, batch_size=16,
                                validation_data=(X_val, y_val), callbacks=[checkpoint, early_stopping])

Epoch 1/500
108/108 [==============================] - 2s 5ms/step - loss: 15.1941 - mae: 11.1813 - val_loss: 5.5404 - val_mae: 2.9449
Epoch 2/500
108/108 [==============================] - 0s 3ms/step - loss: 8.1707 - mae: 6.0119 - val_loss: 3.7195 - val_mae: 1.8418
Epoch 3/500
108/108 [==============================] - 0s 3ms/step - loss: 4.0052 - mae: 2.2800 - val_loss: 2.9580 - val_mae: 1.3690
Epoch 4/500
108/108 [==============================] - 0s 3ms/step - loss: 3.5819 - mae: 2.1119 - val_loss: 2.9777 - val_mae: 1.6193
Epoch 5/500
108/108 [==============================] - 0s 3ms/step - loss: 3.2590 - mae: 1.9961 - val_loss: 2.4659 - val_mae: 1.2961
Epoch 6/500
108/108 [==============================] - 0s 3ms/step - loss: 3.0164 - mae: 1.9305 - val_loss: 2.2531 - val_mae: 1.2467
Epoch 7/500
108/108 [==============================] - 0s 4ms/step - loss: 2.9100 - mae: 1.9698 - val_loss: 2.3091 - val_mae: 1.4325
Epoch 8/500
108/108 [==============================] - 0s 4ms/step 

In [843]:
from sklearn.metrics import mean_squared_error
print('results:')
print('mae: {:2.5}'.format(mean_absolute_error(model3.predict(X_test), y_test)))
print('mse: {:2.5}'.format(mean_squared_error(model3.predict(X_test), y_test)))

results:
mae: 1.3252
mse: 4.7649
